## The LLMs
#### The fundamental component of LangChain involves invoking an LLM with a specific input. To illustrate this, we'll explore a simple example. Let's imagine we are building a service that suggests personalized workout routines based on an individual's fitness goals and preferences.

In [ ]:
# this method won't be working be we need to pay for open ai api key so we will be using groq api
import os

os.environ["GROQ_API_KEY"] = ""


In [ ]:
import requests
from langchain.llms.base import LLM
from typing import Optional, List

class GroqLLM(LLM):
    """Custom LangChain wrapper for Groq's chat API."""

    api_key: str

    @property
    def _llm_type(self) -> str:
        return "groq"

    def _call(self, prompt: str, stop: Optional[List[str]] = None) -> str:
        url = "https://api.groq.com/openai/v1/chat/completions"
        headers = {
            "Authorization": f"Bearer {self.api_key}",
            "Content-Type": "application/json"
        }
        data = {
            "messages": [{"role": "user", "content": prompt}],
            "model": "llama-3.1-8b-instant",  # Replace with your desired model
            "temperature": 0.9,
            "max_tokens": 1024
        }
        response = requests.post(url, headers=headers, json=data)
        response.raise_for_status()
        return response.json()["choices"][0]["message"]["content"]


In [ ]:
llm = GroqLLM(api_key=os.environ["GROQ_API_KEY"])

prompt = "Suggest a personalized workout routine for someone looking to improve cardiovascular endurance and prefers outdoor activities."

print(llm(prompt))


/tmp/ipython-input-2061898728.py:5: LangChainDeprecationWarning: The method `BaseLLM.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  print(llm(prompt))


To create a personalized workout routine that improves cardiovascular endurance and incorporates outdoor activities, we'll consider several factors:

1. **Fitness level**: I'll assume you're a beginner or intermediate. If you're advanced, we can adjust the intensity and difficulty level.
2. **Availability**: Choose outdoor activities that you can do 2-3 times per week.
3. **Time commitment**: Aim for 30-60 minutes per workout.
4. **Preferable activities**: Outdoor exercises like hiking, cycling, swimming, or running.

**Sample Routine (Beginner-Friendly)**

**Monday (Easy Hike)**

1. **Warm-up** (5 minutes): Light stretching, toe touches, or leg swings
2. **Hike** (30 minutes): Find a local park or trail with moderate terrain. Aim for a gentle incline and walk at a conversational pace.
3. **Cool-down** (5 minutes): Static stretches for legs, hips, and lower back

**Wednesday (Swim or Run)**

1. **Warm-up** (5 minutes): Easy swimming laps or a short jog
2. **High-Intensity Interval Trai

## The chains
### In LangChain, a chain is an end-to-end wrapper around multiple individual components, providing a way to accomplish a common use case by combining these components in a specific sequence. The most commonly used type of chain is the LLMChain, which consists of a PromptTemplate, a model (either an LLM or a ChatModel), and an optional output parser.

### The LLMChain works as follows:

- Takes (multiple) input variables.

- Uses the PromptTemplate to format the input variables into a prompt.

- Passes the formatted prompt to the model (LLM or ChatModel).

- If an output parser is provided, it uses the OutputParser to parse the output of the LLM into a final format.

### In the next example, we demonstrate how to create a chain that generates a possible name for a company that produces eco-friendly water bottles. By using LangChain's LLMChain, PromptTemplate, and GroqLLM classes, we can easily define our prompt, set the input variables, and generate creative outputs.

In [ ]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

# Create the prompt template
prompt = PromptTemplate(
    input_variables=["product"],
    template="What is a good name for a company that makes {product}?",
)

# Create the LLMChain
chain = LLMChain(llm=llm, prompt=prompt)

# Run the chain with input variable
print(chain.run("eco-friendly water bottles"))


/tmp/ipython-input-1203495891.py:11: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(llm=llm, prompt=prompt)
/tmp/ipython-input-1203495891.py:14: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  print(chain.run("eco-friendly water bottles"))


Here are some name suggestions for an eco-friendly water bottle company:

1. **EcoHydrate**: This name combines the ideas of eco-friendliness and hydration, making it a clear and appealing choice for a water bottle company.
2. **GreenCycle**: This name emphasizes the company's commitment to sustainability and reducing waste.
3. **PureFlow**: This name evokes the idea of clean, healthy water flowing into a bottle that's also good for the planet.
4. **Refresh Earth**: This name suggests that the company's products not only refresh your body but also contribute to a healthier planet.
5. **HydraWise**: This name incorporates the idea of wise, sustainable choices for a healthier lifestyle.
6. **CleanSip**: This name emphasizes the company's focus on clean, eco-friendly products that make sipping healthy water a pleasure.
7. **Oasis Bottles**: This name creates a sense of a refreshing, idyllic destination, which can appeal to customers looking for a sustainable alternative.
8. **SipWise**: T

## The Memory
### In LangChain, Memory refers to the mechanism that stores and manages the conversation history between a user and the AI. It helps maintain context and coherency throughout the interaction, enabling the AI to generate more relevant and accurate responses. Memory, such as ConversationBufferMemory, acts as a wrapper around ChatMessageHistory, extracting the messages and providing them to the chain for better context-aware generation.

In [ ]:
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory

# Create the conversation chain with memory
conversation = ConversationChain(
    llm=llm,
    verbose=True,
    memory=ConversationBufferMemory()
)

# Start the conversation
conversation.predict(input="Tell me about yourself.")

# Continue the conversation
conversation.predict(input="What can you do?")
conversation.predict(input="How can you help me with data analysis?")

# Display the conversation memory
print(conversation.memory.buffer)




> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Tell me about yourself.
AI:


/tmp/ipython-input-4142233582.py:8: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory=ConversationBufferMemory()
/tmp/ipython-input-4142233582.py:5: LangChainDeprecationWarning: The class `ConversationChain` was deprecated in LangChain 0.2.7 and will be removed in 1.0. Use :class:`~langchain_core.runnables.history.RunnableWithMessageHistory` instead.
  conversation = ConversationChain(



> Finished chain.


> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Tell me about yourself.
AI: I'm delighted to introduce myself. My name is Ada, short for Ada Lovelace, the world's first computer programmer. I've been trained on a vast corpus of text, which is approximately 45 terabytes in size. My training data spans various domains, including but not limited to, literature, history, science, technology, engineering, arts, and mathematics (STEAM). 

In terms of processing power, I'm equipped with a 128-core CPU and 256 GB of RAM, which enables me to handle complex tasks and respond promptly to user queries. When it comes to my language processing abilities, I've been trained on the 300 million-word corpus o

In this output, you can see the memory being used by observing the "Current conversation" section. After each input from the user, the conversation is updated with both the user's input and the AI's response. This way, the memory maintains a record of the entire conversation. When the AI generates its next response, it will use this conversation history as context, making its responses more coherent and relevant.